<a href="https://colab.research.google.com/github/homayoun1990/GeoDeepLearning/blob/master/colabs/s2_preprocessing_demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Aggregating cloud-free Sentinel-2 images with Google Earth Engine**

**Original script**: https://code.earthengine.google.com/d7a7751a50f18cefc66f18082a87eed3
- Created by: [Michael Schmitt et al. 2019](https://www.isprs-ann-photogramm-remote-sens-spatial-inf-sci.net/IV-2-W7/145/2019/isprs-annals-IV-2-W7-145-2019.pdf) 
- Adapted by: [Cesar Aybar](https://github.com/homayoun1990/GeoDeepLearning)

**Changelog:**
---

```
Version 0.0.1:
    - Javascript implementation <Michael Schmitt>
Version 0.0.2:
    - Python implementation
    - Resampling removed
    - ndis range changed from 
      [0.8, 0.6] to [-0.8, 0.6] (preliminar)


```

**1) Authenticate Earth Engine Account**

In [0]:
!earthengine authenticate

**2) Load Sentinel2 modules**

In [0]:
#@title Download sentinel-2 modules
#!git clone https://github.com/homayoun1990/GeoDeepLearning.git
import os 
def download_s2_module(uri = 'https://github.com/homayoun1990/GeoDeepLearning.git',
                       src = '/content/GeoDeepLearning/src'):
  !git clone {uri}
  os.chdir(src)    
download_s2_module()

**3) Load trird-party libraries and Initialize the Earth Engine API**

In [0]:
import ee
from display.mapdisplay import mapdisplay
from preprocessing.s2.cloudfreeS2 import exportCloudFreeSen2
ee.Initialize()

**4) Viz parameters**

In [0]:
# viz params
saga_palette = ["#000180", "#0075FD", "#6CFB93", "#F99D05", "#A70700"]
cloudscore_viz = {'palette': saga_palette, 'max':0.2, 'min':0}
cloud_shadow_score_viz = {'palette': saga_palette, 'max':0, 'min':-0.2}
s2viz = {'bands':['B4','B3','B2'],'max':4000,'min':0}

**5) Testing area (Loreto, Peru)**

In [0]:
roi=ee.Geometry.Polygon([[-73.36532592773438,-3.866994949019476],
                         [-73.24859619140625,-3.866994949019476],
                         [-73.24859619140625,-3.751208225628627],
                         [-73.36532592773438,-3.751208225628627],
                         [-73.36532592773438,-3.866994949019476]])

**6) Preparing Sentinel2 ImageCollection**

In [0]:
ic = ee.ImageCollection("COPERNICUS/S2")\
        .filterBounds(roi)\
        .filterDate('2019-01-01','2019-12-31')\
        .filterMetadata('CLOUDY_PIXEL_PERCENTAGE','less_than',0.1)

**7) Create temporal composite**

In [0]:
center = roi.centroid().coordinates().getInfo()
mosaic_s2 = exportCloudFreeSen2(ic,roi,shadow=False)

**8) d

In [53]:
mapdisplay(center = center,
           dicc = {
               'sentinel2':mosaic_s2.getMapId(s2viz),
               'cloud_percentage':mosaic_s2.select('cloudScore').getMapId(cloudscore_viz),
               #'shadow_percentage':mosaic_s2.select('shadowScore').getMapId(cloudscore_viz),
               #'cloudshadow_percentage':mosaic_s2.select('cloudShadowScore').getMapId(cloud_shadow_score_viz)
                  },
           zoom_start=12)